In [38]:
import pandas as pd
import geopandas as gpd
import folium
import branca
import json
from shapely.geometry import Polygon, MultiPolygon, Point
import matplotlib.pyplot as plt
import io
import requests

In [39]:
#GEODATA ONLINE
df_deptos = gpd.read_file('https://www.dropbox.com/s/y30ylt8qaz516yu/MGN_DPTO_POLITICO.geojson?dl=1',driver='GeoJSON')
s = requests.get("https://www.dropbox.com/s/0wu5t06rfuyyebo/mintic_investments_clean.csv?dl=1").content
df_investment = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [40]:
#order_info by department
df_investment=df_investment.sort_values(by=['departame_cod'],ascending=True)
df_investment_total=df_investment.groupby(['anio_corte','departame_cod','viceministerio'])['inversion'].agg('sum').reset_index()
df_investment_total.head(15)

,anio_corte,departame_cod,viceministerio,inversion
0,1990,11,transformacion,0
1,1990,13,transformacion,0
2,1990,15,transformacion,0
3,1990,17,transformacion,0
4,1990,18,transformacion,0
5,1990,19,transformacion,0
6,1990,20,transformacion,0
7,1990,23,transformacion,0
8,1990,25,transformacion,0
9,1990,41,transformacion,0


In [42]:
#CLEAN DATA
df_investment_total.loc[df_investment_total['departame_cod'].str.len()==1,'departame_cod'] = '0' + df_investment_total['departame_cod'].astype(str)
#MERGE DATA
df_merge = pd.merge(df_investment_total, df_deptos,  how='left', left_on=['departame_cod'], right_on = ['DPTO_CCDGO'])
print(df_merge)

     anio_corte departame_cod  viceministerio    inversion DPTO_CCDGO  \
0          1990            11  transformacion            0         11   
1          1990            13  transformacion            0         13   
2          1990            15  transformacion            0         15   
3          1990            17  transformacion            0         17   
4          1990            18  transformacion            0         18   
..          ...           ...             ...          ...        ...   
707        2021            95  transformacion            0         95   
708        2021            97    conectividad   4785206481         97   
709        2021            97  transformacion            0         97   
710        2021            99    conectividad  12470538102         99   
711        2021            99  transformacion            0         99   

       DPTO_CNMBR  DPTO_ANO_C                                  DPTO_ACT_A  \
0    BOGOTÁ, D.C.      1538.0               Co

In [46]:
##DEPARTMENTS
df_merge.dropna()
df_merge.sort_values(by=['anio_corte','viceministerio'],ascending=False)
print(df_merge)
df_automation=df_merge.copy()


     anio_corte departame_cod  viceministerio    inversion DPTO_CCDGO  \
0          1990            11  transformacion            0         11   
1          1990            13  transformacion            0         13   
2          1990            15  transformacion            0         15   
3          1990            17  transformacion            0         17   
4          1990            18  transformacion            0         18   
..          ...           ...             ...          ...        ...   
707        2021            95  transformacion            0         95   
708        2021            97    conectividad   4785206481         97   
709        2021            97  transformacion            0         97   
710        2021            99    conectividad  12470538102         99   
711        2021            99  transformacion            0         99   

       DPTO_CNMBR  DPTO_ANO_C                                  DPTO_ACT_A  \
0    BOGOTÁ, D.C.      1538.0               Co

In [8]:
df_merge=geo_per_year_vice(df_merge,2021,'conectividad')

In [9]:
## GET MAX MIN VALUE INVESTMENT PER VICEMINISTRY PER YEAR
column = df_merge["inversion"]
max_value = column.max()
#print(max_value)
min_value = column.min()
#print(min_value)

In [10]:
#style specification
colormap = branca.colormap.LinearColormap(
    colors=['white','yellow','orange','red','darkred'],
    vmin=min_value, # min_value 0
    vmax=max_value # max_value 35
)

In [11]:
style_function = lambda x: {
    'fillColor': colormap(x['properties']['INVESTMENT']), #VARIABLE SHOULD BE INVESTMENT
    'color': 'black',
    'weight':2,
    'fillOpacity':0.5
}

In [144]:
def iterator(object,cod,nom,var,var1,var2):
    for obj in object:
        for attribute, value in obj.items():
            if "properties" in attribute:
                obj['properties']={"DPTO_CCDGO": cod, "DPTO_CNMBR": nom, "INVESTMENT": var, "VICEMINISTRY":var1, "YEAR":var2} #VARIABLE SHOULD BE INVESTMENT
    return object

In [ ]:
map = folium.Map(location=[4.714331, -74.112808], zoom_start=5 , tiles='CartoDB positron')#tiles="Stamen Terrain"
#REMOVE INVALID DATA 
df_merge = df_merge[df_merge.geometry != None]

for _, r in df_merge.iterrows():  #df_deptos.iterrows() ITERATE df_merge
    #without simplifying the representation of each borough, the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    t = json.loads(geo_j)
    t['features']=iterator(t['features'],r['DPTO_CCDGO'],r['DPTO_CNMBR'],r['inversion'],r['viceministerio'],r['anio_corte']) #VARIABLE SHOULD BE INVESTMENT
    geo_j=t
    
    #dep=geo_j['features'][0]['properties']['DPTO_CNMBR']
    #var=geo_j['features'][0]['properties']['VARIABLE']
    
    geo_j = folium.GeoJson(data=geo_j,style_function=style_function,tooltip=folium.GeoJsonTooltip(
                           fields=['DPTO_CNMBR','INVESTMENT','VICEMINISTRY','YEAR'],
                           aliases=['Department', 'Invesment','Viceministry','Year'], 
                           localize=True)
            )  
    #folium.Popup(r['DPTO_CNMBR']).add_to(geo_j)
    geo_j.add_to(map)
    #STILL STATIC
map.save("data/conectividad2021.html")    
map

In [ ]:
def gen_map(df_merge,year,vice):
    map = folium.Map(location=[4.714331, -74.112808], zoom_start=5 , tiles='CartoDB positron')#tiles="Stamen Terrain"
    #REMOVE INVALID DATA 
    df_merge = df_merge[df_merge.geometry != None]

    for _, r in df_merge.iterrows():  #df_deptos.iterrows() ITERATE df_merge
        #without simplifying the representation of each borough, the map might not be displayed
        sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
        geo_j = sim_geo.to_json()
        t = json.loads(geo_j)
        t['features']=iterator(t['features'],r['DPTO_CCDGO'],r['DPTO_CNMBR'],r['inversion'],r['viceministerio'],r['anio_corte']) #VARIABLE SHOULD BE INVESTMENT
        geo_j=t
    
        #dep=geo_j['features'][0]['properties']['DPTO_CNMBR']
        #var=geo_j['features'][0]['properties']['VARIABLE']
        
        
        column = df_merge["inversion"]
        max_value = column.max()
        #print(max_value)
        min_value = column.min()
        #print(min_value)
        #style specification
        colormap = branca.colormap.LinearColormap(
        colors=['white','yellow','orange','red','darkred'],
        vmin=min_value, # min_value 0
        vmax=max_value # max_value 35
        )     
        
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {
                                                         'fillColor': colormap(x['properties']['INVESTMENT']), #VARIABLE SHOULD BE INVESTMENT
                                                         'color': 'black',
                                                         'weight':2,
                                                         'fillOpacity':0.5},
                               tooltip=folium.GeoJsonTooltip(
                               fields=['DPTO_CNMBR','INVESTMENT','VICEMINISTRY','YEAR'],
                               aliases=['Department', 'Invesment','Viceministry','Year'], 
                               localize=True)
            )  
        #folium.Popup(r['DPTO_CNMBR']).add_to(geo_j)
        geo_j.add_to(map)
    #STILL STATIC
    #map.save("data/conectividad2021.html")   
    string = "data/"+vice+str(year)+".html"
    map.save(string)  
    map  

In [148]:
#FUNCTION TO GET DATA FROM DEPARMENT, YEAR, AND VICEMINISTRY
def geo_per_year_vice(d,year,vice):
    df=pd.DataFrame(data=d)
    df=df.loc[(df['anio_corte'] == year) & (df['viceministerio'] == vice)]
    ##update_colormap(df)
    return df 

In [149]:
def automate_process(df,year,viceministry):
    for y in year:
        for v in viceministry:
            df_map=geo_per_year_vice(df,y,v)
            gen_map(df_map,y,v)   

In [150]:
def update_colormap(df):
    ## GET MAX MIN VALUE INVESTMENT PER VICEMINISTRY PER YEAR
    column = df["inversion"]
    max_value = column.max()
    #print(max_value)
    min_value = column.min()
    #print(min_value)
    #style specification
    colormap = branca.colormap.LinearColormap(
    colors=['white','yellow','orange','red','darkred'],
    vmin=min_value, # min_value 0
    vmax=max_value # max_value 35
    )  

In [151]:
##AUTOMATIC GENERATION

year = [2016,2017,2018,2019,2020,2021]
viceministry=['conectividad','transformacion']        
automate_process(df_automation,year,viceministry)             

53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
53936290714
670270394
52829438440
194935163
52829438440
194935163
52829438440
194935163
52829438440
194935163
52829438440
194935163
52829438440
194935163
52829438440
194935163
52829438440
194935163
52829438440
194935163
52829438440
194935163
52829438440
194935163
52829438440
194935163
52829438440
194935163
52829438440
194935163
52829438440
194935163
52829438440
194935163
5282943844